# Projet SDP (Mention IA)

## Question 1 : Explication de type (1-1)

### Prologue
Nous nous plaçons dans l’exemple de comparaison entre deux candidats (ici notés `x` et `y`) évalués sur des cours $A\dots G$ avec des poids, et classés par **somme pondérée**.

### Données (x et y)  

On encode les notes suivantes (sur 100) et les poids :

- Critères : $A,B,C,D,E,F,G$
- Candidat $x$ : 85, 81, 71, 69, 75, 81, 88  
- Candidat $y$ : 81, 81, 75, 63, 67, 88, 95  
- Poids : 8, 7, 7, 6, 6, 5, 6

On définit la **contribution** de chaque critère $k$ à l’affirmation $x \succ y$ par : $d_k = w_k \,(x_k - y_k)$
- $d_k>0$ : critère **pour** $x\succ y$ (ensemble `pros`)  
- $d_k<0$ : critère **contre** $x\succ y$ (ensemble `cons`)  
- $d_k=0$ : critère **neutre**


In [1]:
# Données de l'exemple (x ≻ y)
criteres = ["A","B","C","D","E","F","G"]

notes_x = {"A":85,"B":81,"C":71,"D":69,"E":75,"F":81,"G":88}
notes_y = {"A":81,"B":81,"C":75,"D":63,"E":67,"F":88,"G":95}

poids = {"A":8,"B":7,"C":7,"D":6,"E":6,"F":5,"G":6}

# Contributions d_k = w_k (x_k - y_k)
d = {k: poids[k]*(notes_x[k]-notes_y[k]) for k in criteres}
d


{'A': 32, 'B': 0, 'C': -28, 'D': 36, 'E': 48, 'F': -35, 'G': -42}

### Formulation : ensembles pros/cons/neutres
On calcule automatiquement les ensembles `pros(x,y)`, `cons(x,y)` et `neutral(x,y)` à partir des contributions $d_k$.

In [2]:
pros = [k for k in criteres if d[k] > 0]
cons = [k for k in criteres if d[k] < 0]
neutral = [k for k in criteres if d[k] == 0]

print("Contributions d_k :")
for k in criteres:
    print(f"  {k} : {d[k]:+d}")

print("\npros(x,y)   =", pros)
print("cons(x,y)   =", cons)
print("neutral(x,y)=", neutral)


Contributions d_k :
  A : +32
  B : +0
  C : -28
  D : +36
  E : +48
  F : -35
  G : -42

pros(x,y)   = ['A', 'D', 'E']
cons(x,y)   = ['C', 'F', 'G']
neutral(x,y)= ['B']


### Trade-offs (1-1)

Un trade-off de type **(1-1)** est une paire $(P,C)$ avec :
- $P \in \mathrm{pros}(x,y)$
- $C \in \mathrm{cons}(x,y)$
- et $d_P + d_C > 0$

On peut donc lister toutes les paires admissibles $(P,C)$ par simple énumération.


In [3]:
tradeoffs_11 = []
for p in pros:
    for c in cons:
        if d[p] + d[c] > 0:
            tradeoffs_11.append((p,c,d[p]+d[c]))

print("Trade-offs (1-1) admissibles (P,C) tels que d_P + d_C > 0 :")
for (p,c,val) in sorted(tradeoffs_11, key=lambda t: (-t[2], t[0], t[1])):
    print(f"  ({p},{c}) : d_{p}+d_{c} = {val:+d}")

Trade-offs (1-1) admissibles (P,C) tels que d_P + d_C > 0 :
  (E,C) : d_E+d_C = +20
  (E,F) : d_E+d_F = +13
  (D,C) : d_D+d_C = +8
  (E,G) : d_E+d_G = +6
  (A,C) : d_A+d_C = +4
  (D,F) : d_D+d_F = +1


### Formulation par optimisation

#### Variables de décision
Pour chaque paire admissible $(p,c)$, on introduit une variable binaire :
$$
x_{p,c} \in \{0,1\}
$$
avec $x_{p,c}=1$ si on retient le trade-off $(p,c)$.

#### Contraintes
1) **Chaque critère contre** doit être compensé exactement une fois :
$$
\forall c \in \mathrm{cons}(x,y),\quad \sum_{p \in \mathrm{pros}(x,y)} x_{p,c} = 1
$$

2) **Disjonction** : un même critère pour ne peut servir qu’une fois :
$$
\forall p \in \mathrm{pros}(x,y),\quad \sum_{c \in \mathrm{cons}(x,y)} x_{p,c} \le 1
$$

3) Variables définies **uniquement** pour les paires admissibles $(p,c)$ où $d_p + d_c > 0$.

#### Objectif
On n'a rien à miniser, alors l'objectif sera $0$ (un problème de faisabilité).

#### Certificat de non-existence
Si le modèle est **infaisable**, on demande à Gurobi un **IIS** (*Irreducible Inconsistent Subsystem*) : un sous-ensemble minimal de contraintes incompatible. C’est un **certificat de non‑existence** exploitable dans un rendu.


In [4]:
# Implémentation avec Gurobi
# - Si le modèle est faisable : on extrait une explication (1-1)
# - Sinon : on calcule un IIS comme certificat de non-existence

import gurobipy as gp
from gurobipy import GRB

# Construction de l'ensemble des paires admissibles
def explication_1_1(d, pros, cons, neutral):
    A = [(p,c) for p in pros for c in cons if d[p] + d[c] > 0]

    m = gp.Model("explication_1_1")

    # Variables binaires x[p,c] pour (p,c) admissible
    xvar = m.addVars(A, vtype=GRB.BINARY, name="x")

    # Chaque 'contre' doit être couvert exactement une fois
    c_cover = {}
    for c in cons:
        c_cover[c] = m.addConstr(gp.quicksum(xvar[p,c] for p in pros if (p,c) in xvar) == 1,
                                 name=f"couvrir_cons[{c}]")

    # Chaque 'pour' utilisé au plus une fois (disjonction côté pros)
    p_use = {}
    for p in pros:
        p_use[p] = m.addConstr(gp.quicksum(xvar[p,c] for c in cons if (p,c) in xvar) <= 1,
                               name=f"utiliser_pros_au_plus_une_fois[{p}]")

    # Objectif : trouver une solution faisable
    m.setObjective(0, GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()

    if m.Status == GRB.OPTIMAL:
        print("\n Une explication (1-1) existe. Paires sélectionnées :\n")
        solution = [(p,c) for (p,c) in A if xvar[p,c].X > 0.5]
        for (p,c) in solution:
            print(f"  - ({p},{c}) : l'avantage de x sur y en {p}({d[p]:+d}) pèse plus fort que son désavantage en {c}({d[c]:+d})  => somme={d[p]+d[c]:+d}")
        for n in neutral:
            print(f"  - x et y se valent en {n}")
        print(f"\nLongueur l = {len(solution)}")


    elif m.Status == GRB.INFEASIBLE:
        print("\n Aucune explication (1-1) n'existe : modèle infaisable.")
        print("Calcul d'un IIS (certificat de non-existence) ...")
        m.computeIIS()
        print("\nContraintes appartenant à l'IIS :")
        for constr in m.getConstrs():
            if constr.IISConstr:
                print(" -", constr.ConstrName)
    else:
        print("\nStatut inattendu :", m.Status)

explication_1_1(d, pros, cons, neutral)

Set parameter Username
Set parameter LicenseID to value 2754485
Academic license - for non-commercial use only - expires 2026-12-12

 Une explication (1-1) existe. Paires sélectionnées :

  - (A,C) : l'avantage de x sur y en A(+32) pèse plus fort que son désavantage en C(-28)  => somme=+4
  - (D,F) : l'avantage de x sur y en D(+36) pèse plus fort que son désavantage en F(-35)  => somme=+1
  - (E,G) : l'avantage de x sur y en E(+48) pèse plus fort que son désavantage en G(-42)  => somme=+6
  - x et y se valent en B

Longueur l = 3


## Question 2 : Explication de type (1-m)

### Données

In [5]:
criteres = ["A", "B", "C", "D", "E", "F", "G"]

notes = {
    "x":  {"A": 85, "B": 81, "C": 71, "D": 69, "E": 75, "F": 81, "G": 88},
    "y":  {"A": 81, "B": 81, "C": 75, "D": 63, "E": 67, "F": 88, "G": 95},
    "z":  {"A": 74, "B": 89, "C": 74, "D": 81, "E": 68, "F": 84, "G": 79},
    "t":  {"A": 74, "B": 71, "C": 84, "D": 91, "E": 77, "F": 76, "G": 73},
    "u":  {"A": 72, "B": 75, "C": 66, "D": 85, "E": 88, "F": 66, "G": 93},
    "v":  {"A": 71, "B": 73, "C": 63, "D": 92, "E": 76, "F": 79, "G": 93},
    "w":  {"A": 79, "B": 69, "C": 78, "D": 76, "E": 67, "F": 84, "G": 79},
    "w'": {"A": 57, "B": 76, "C": 81, "D": 76, "E": 82, "F": 86, "G": 77},
}

weights = {"A": 8, "B": 7, "C": 7, "D": 6, "E": 6, "F": 5, "G": 6}

### Comparaison $w \succ w'$

In [6]:
contributions = [weights[k] * (notes["w"][k] - notes["w'"][k]) for k in criteres]
contributions

[176, -49, -21, 0, -90, -10, 12]

On a plus de "contre" que de "pour", alors on ne peut pas utliser un trade-off type (1-1)

### Modélisation type (1-m)

#### 1. Variables de décision
Contrairement au cas (1-1), un argument *pour* peut maintenant couvrir plusieurs arguments *contre*.
Nous définissons une variable binaire qui associe un argument contre $c$ à un argument pour $p$ :

$$
x_{p,c} = 
\begin{cases} 
1 & \text{si l'argument } c \in \text{cons} \text{ est expliqué par } p \in \text{pros} \\
0 & \text{sinon}
\end{cases}
$$

Cette modélisation permet de construire implicitement des groupes $\mathcal{C}_p = \{c \mid x_{p,c} = 1\}$ associés à chaque $p$.

#### 2. Contraintes

**C1 : Partition des arguments "Contre" (Couverture)** Chaque argument négatif $c \in \text{cons}(x,y)$ doit appartenir à exactement un trade-off (il doit être expliqué par un seul $p$).
$$
\sum_{p \in \text{pros}} x_{p,c} = 1, \quad \forall c \in \text{cons}
$$

**C2 : Validité des Trade-offs (Positivité)** Pour chaque argument $p \in \text{pros}$, si celui-ci est utilisé pour expliquer un groupe de défauts, la somme des contributions doit être strictement positive.
$$
d_p + \sum_{c \in \text{cons}} d_c \cdot x_{p,c} \ge 1, \quad \forall p \in \text{pros}
$$
*Note : Si un argument $p$ n'est pas utilisé (tous les $x_{p,c}=0$), l'inégalité devient $d_p \ge 1$, ce qui est toujours vrai car $p \in \text{pros}$. Si $p$ est utilisé, cela garantit que l'avantage compense le cumul des désavantages.*

#### 3. Objectif
Comme précédemment, il s'agit d'un problème de satisfaction (trouver si une explication existe). La fonction objectif est constante ou nulle.
$$
\text{Minimiser } 0
$$

In [7]:
def explication_1_m(weights, notes, criteres, i1, i2):
    # Construction de l'ensemble des paires admissibles
    d = {k: weights[k]*(notes[i1][k]-notes[i2][k]) for k in criteres}
    pros = [k for k in criteres if d[k] > 0]
    cons = [k for k in criteres if d[k] < 0]
    neutral = [k for k in criteres if d[k] == 0]
    A = [(p,c) for p in pros for c in cons]

    m = gp.Model("explication_1_m")

    # Variables binaires x[p,c] pour (p,c) admissible
    xvar = m.addVars(A, vtype=GRB.BINARY, name="x")

    # Chaque 'contre' doit être couvert exactement une fois
    c_cover = {}
    for c in cons:
        c_cover[c] = m.addConstr(gp.quicksum(xvar[p,c] for p in pros if (p,c) in xvar) == 1,
                                 name=f"couvrir_cons[{c}]")

    # Chaque 'pour' utilisé au plus une fois (disjonction côté pros)
    p_use = {}

    # d[p] + sum_{c} x[p,c]*d[c] >= 1
    for p in pros:
        p_use[p] = m.addConstr(d[p] + gp.quicksum(d[c]*xvar[p,c] for c in cons if (p,c) in xvar) >= 1,
                               name=f"utiliser_pros_au_plus_une_fois[{p}]")

    # Objectif : trouver une solution faisable
    m.setObjective(0, GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()

    if m.Status == GRB.OPTIMAL:
        print("\n Une explication (1-m) existe. Groupes sélectionnés :\n")
        solution = [(p, c) for (p, c) in A if xvar[p, c].X > 0.5]

        # Regroupement par p
        groupes = {}
        for p, c in solution:
            groupes.setdefault(p, []).append(c)

        for p, cs in groupes.items():
            cs_list = ", ".join(cs)
            somme_c = sum(d[c] for c in cs)
            total = d[p] + somme_c
            print(
                f"  - ({p}, {{{cs_list}}}) : " +
                f"l'avantage en {p} ({d[p]:+d}) et désavantage en {{{cs_list}}} " +
                f"({somme_c:+d}) => somme={total:+d}"
            )

        for n in neutral:
            print(f"  - x et y se valent en {n}")

        print(f"\nLongueur l = {len(groupes)}")
    elif m.Status == GRB.INFEASIBLE:
        print("\n Aucune explication (1-m) n'existe : modèle infaisable.")
        print("Calcul d'un IIS (certificat de non-existence) ...")
        m.computeIIS()
        print("\nContraintes appartenant à l'IIS :")
        for constr in m.getConstrs():
            if constr.IISConstr:
                print(" -", constr.ConstrName)
    else:
        print("\nStatut inattendu :", m.Status)

explication_1_m(weights, notes, criteres, "w", "w'")


 Une explication (1-m) existe. Groupes sélectionnés :

  - (A, {B, C, E, F}) : l'avantage en A (+176) et désavantage en {B, C, E, F} (-170) => somme=+6
  - x et y se valent en D

Longueur l = 1


In [8]:
explication_1_m(weights, notes, criteres, "u", "v")


 Aucune explication (1-m) n'existe : modèle infaisable.
Calcul d'un IIS (certificat de non-existence) ...

Contraintes appartenant à l'IIS :
 - couvrir_cons[D]
 - couvrir_cons[F]
 - utiliser_pros_au_plus_une_fois[A]
 - utiliser_pros_au_plus_une_fois[B]
 - utiliser_pros_au_plus_une_fois[C]
 - utiliser_pros_au_plus_une_fois[E]


## Question 3 : Explication de type (m-1)

In [16]:
def explication_m_1(weights, notes, criteres, i1, i2):
    # Construction de l'ensemble des paires admissibles
    d = {k: weights[k]*(notes[i1][k]-notes[i2][k]) for k in criteres}
    pros = [k for k in criteres if d[k] > 0]
    cons = [k for k in criteres if d[k] < 0]
    neutral = [k for k in criteres if d[k] == 0]

    A = [(p,c) for p in pros for c in cons]

    m = gp.Model("explication_m_1")

    # Variables binaires x[p,c] pour (p,c) admissible
    xvar = m.addVars(A, vtype=GRB.BINARY, name="x")

    # Chaque 'contre' doit être couvert au moins une fois
    # d[c] + sum_{p} x[p,c]*d[p] >= 1
    c_cover = {}
    for c in cons:
        c_cover[c] = m.addConstr(d[c] + gp.quicksum(d[p]*xvar[p,c] for p in pros if (p,c) in xvar) >= 1,
                                 name=f"couvrir_cons[{c}]")

    # Chaque 'pour' utilisé au plus une fois (disjonction côté pros)
    p_use = {}
    for p in pros:
        p_use[p] = m.addConstr(gp.quicksum(xvar[p,c] for c in cons if (p,c) in xvar) <= 1,
                               name=f"utiliser_pros_au_plus_une_fois[{p}]")

    # Objectif : trouver une solution faisable
    m.setObjective(0, GRB.MINIMIZE)
    m.params.outputflag = 0
    m.optimize()

    if m.Status == GRB.OPTIMAL:
        print("\n Une explication (m-1) existe. Groupes sélectionnés :\n")
        solution = [(p, c) for (p, c) in A if xvar[p, c].X > 0.5]

        # Regroupement par c
        groupes = {}
        for p, c in solution:
            groupes.setdefault(c, []).append(p)

        for c, ps in groupes.items():
            ps_list = ", ".join(ps)
            somme_p = sum(d[p] for p in ps)
            total = d[c] + somme_p
            print(
                f"  - ({{{ps_list}}}, {c}) : " +
                f"l'avantage en {c} ({d[c]:+d}) et désavantage en {{{ps_list}}} " +
                f"({somme_p:+d}) => somme={total:+d}"
            )

        for n in neutral:
            print(f"  - x et y se valent en {n}")

        print(f"\nLongueur l = {len(groupes)}")
    elif m.Status == GRB.INFEASIBLE:
        print("\n Aucune explication (m-1) n'existe : modèle infaisable.")
        print("Calcul d'un IIS (certificat de non-existence) ...")
        m.computeIIS()
        print("\nContraintes appartenant à l'IIS :")
        for constr in m.getConstrs():
            if constr.IISConstr:
                print(" -", constr.ConstrName)
    else:
        print("\nStatut inattendu :", m.Status)

explication_m_1(weights, notes, criteres, "u", "v")


 Une explication (m-1) existe. Groupes sélectionnés :

  - ({A, B, C}, D) : l'avantage en D (-42) et désavantage en {A, B, C} (+43) => somme=+1
  - ({E}, F) : l'avantage en F (-65) et désavantage en {E} (+72) => somme=+7
  - x et y se valent en G

Longueur l = 2


In [17]:
explication_m_1(weights, notes, criteres, "y", "z")


 Aucune explication (m-1) n'existe : modèle infaisable.
Calcul d'un IIS (certificat de non-existence) ...

Contraintes appartenant à l'IIS :
 - couvrir_cons[B]
 - couvrir_cons[D]
 - couvrir_cons[E]
 - utiliser_pros_au_plus_une_fois[A]
 - utiliser_pros_au_plus_une_fois[C]
 - utiliser_pros_au_plus_une_fois[F]
 - utiliser_pros_au_plus_une_fois[G]


In [18]:
explication_m_1(weights, notes, criteres, "z", "t")


 Aucune explication (m-1) n'existe : modèle infaisable.
Calcul d'un IIS (certificat de non-existence) ...

Contraintes appartenant à l'IIS :
 - couvrir_cons[C]
 - couvrir_cons[D]
 - couvrir_cons[E]
 - utiliser_pros_au_plus_une_fois[B]
 - utiliser_pros_au_plus_une_fois[G]


In [19]:
explication_1_m(weights, notes, criteres, "z", "t")


 Aucune explication (1-m) n'existe : modèle infaisable.
Calcul d'un IIS (certificat de non-existence) ...

Contraintes appartenant à l'IIS :
 - couvrir_cons[C]
 - couvrir_cons[D]
 - utiliser_pros_au_plus_une_fois[B]
 - utiliser_pros_au_plus_une_fois[F]
 - utiliser_pros_au_plus_une_fois[G]


## Question 3 : Explication de type (m-1) et (1-m)